In [2]:
import openai
import os

In [3]:
openai.api_type = "azure"
openai.api_version = "2023-05-15"
openai.api_key = "a5637d1b2ad34453807c8a71cecd9dfd"
openai.api_base ="https://jp-sandbox.openai.azure.com/"

In [17]:
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_BASE"] = "https://jp-sandbox.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = "a5637d1b2ad34453807c8a71cecd9dfd"
os.environ["OPENAI_API_VERSION"] = "2023-05-15"
deployment_name = 'gpt-4'

Test if api works

In [138]:
response = openai.ChatCompletion.create(
    engine="gpt-4", # The deployment name you chose when you deployed the GPT-35-Turbo or GPT-4 model.
    messages = [
        {"role": "system", "content": "Assistant is a large language model trained by OpenAI."},
        {"role": "user", "content": "What is EY?"}
        ]
        )

#print(response)
print(response['choices'][0]['message']['content'])

EY, formerly known as Ernst & Young, is one of the "Big Four" global professional services firms. It offers a wide range of services, including audit, tax, consulting, and advisory services to various industries around the world. EY is headquartered in London, England, and has offices in more than 150 countries, employing over 300,000 people.

The company's clients include corporations, governments, and public sector entities. EY helps organizations manage risk, improve performance, and address complex issues by providing tailored solutions. As a member of the Big Four, EY is considered one of the largest and most prestigious accounting and professional services firms in the world, alongside Deloitte, KPMG, and PwC.


**Load data**

In [5]:
# import pandas as pd
# raw_data = pd.read_csv("patient_notes.csv")
# rand_sample = raw_data.sample(n=100, random_state=123)
# rand_sample.shape

In [6]:
#rand_sample.to_csv("random_100notes.csv", index=False)

In [139]:
from langchain.document_loaders.csv_loader import CSVLoader
loader = CSVLoader(
    file_path="./random_100notes.csv",
    csv_args = {"fieldnames": ["pn_num", "case_num", "pn_history"],},
    source_column = "pn_num"
    )

data = loader.load()

In [140]:
data[1]

Document(page_content="pn_num: 81630\ncase_num: 8\npn_history: 67 yo female with hx of breast CA in remission and HTN presents to clinic with c/o 3 weeks of insomnia. Pt states her son died suddenly in late July in an MVA. Since that time, she has found it difficult to fall asleep and also feels as though she tosses and turns and awakens early. She tried a friend's Ambien for the past 5 nights without relief. Of note, pt has felt sad and notes decreased interest in activities as well as low energy. However, she denies SI, HI, depression/hopelessness, guilt, concentration defitict, or psychomotor agitation or depression. Denies palpitations, chest pain, cough, SOB, HA, numbness, tingling or weakness. \r\nROS: as above\r\nPMH: HTN, breast cancer in remission x10 years\r\nFam: Mother hx of depression, in 90s now. No hx of thyroid \r\nSurg: Lumpectomy, laparotomy for appendix rupture in 20s\r\nSoc: Never smoker. 2 glasses of wine 2-3x per week, none in past 3 weeks. No illicit drugs Meds: 

Split the files into chunks

In [141]:
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    deployment = "text-embedding-ada-002",
    model = "gpt-4",
    openai_api_key = os.getenv("OPENAI_API_KEY"),
    openai_api_base = os.getenv("OPENAI_API_BASE"),
    openai_api_type = "azure",
    chunk_size = 16,
)

In [142]:
from langchain.vectorstores import FAISS
#chroma, pinecone 

full_index = FAISS.from_documents(data, embeddings)

Enter Patient Number

In [136]:
#full_index.docstore._dict

In [143]:
data[1]

Document(page_content="pn_num: 81630\ncase_num: 8\npn_history: 67 yo female with hx of breast CA in remission and HTN presents to clinic with c/o 3 weeks of insomnia. Pt states her son died suddenly in late July in an MVA. Since that time, she has found it difficult to fall asleep and also feels as though she tosses and turns and awakens early. She tried a friend's Ambien for the past 5 nights without relief. Of note, pt has felt sad and notes decreased interest in activities as well as low energy. However, she denies SI, HI, depression/hopelessness, guilt, concentration defitict, or psychomotor agitation or depression. Denies palpitations, chest pain, cough, SOB, HA, numbness, tingling or weakness. \r\nROS: as above\r\nPMH: HTN, breast cancer in remission x10 years\r\nFam: Mother hx of depression, in 90s now. No hx of thyroid \r\nSurg: Lumpectomy, laparotomy for appendix rupture in 20s\r\nSoc: Never smoker. 2 glasses of wine 2-3x per week, none in past 3 weeks. No illicit drugs Meds: 

In [169]:
data[2]

Document(page_content="pn_num: 36193\ncase_num: 3\npn_history: Px is 35 yo M presenting with epigastric pain 2 mo's duration.\r\n-Burning, gnawing\r\n-Intermittent, no associated with food intake\r\n-Previous 1x week, now 2/3x day, worsening\r\n-Antacids previously alleviated pain\r\n+fatigue, decreased appetite, bloating\r\n\r\nROS: neg except as above\r\nPMH: back pain\r\nPSH: none\r\nAllergies: NKDA\r\nMeds: motrin, antacid\r\nSH: 1 PPD since 15 yo, 3-4x beers /week, fast food diet\r\nFH: bleeding ulcer - father", metadata={'source': '36193', 'row': 2})

In [144]:
data[100]

Document(page_content='pn_num: 22006\ncase_num: 2\npn_history: 44 yof presents with irregular periods. Patient states it started 3 yrs ago now her last menses was 2 months ago using 2-3 pads and at times 6 pads when heavy. she denies any discharge, odor, bleeding between menses, change in weight, feeling cold or hot, diarrehea or constipation, abdominal pain change in urine or bowel but complains of sweating and hot flashes recently.\r\nROS: negative except as above\r\nPMHX:HTN x 6 yrs, FMHX: mother with OA, meds: HCTZ, NKDA, PSH: none, SH: denies smoke, drugs, social drinker\r\nOBGYN: G2P2, sexually active with husband, denies any STD. Last pap smear was 1 year ago and normal.', metadata={'source': '22006', 'row': 100})

Retrieve rows for this patient only, from the vector store

In [270]:
pat_num = '81630'

# Retrieve rows for this patient only, from the vector store
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import RetrievalQA
retriever = full_index.as_retriever(search_type="mmr", search_kwargs={'filter': {'source': pat_num}}) 
#retriever = full_index.as_retriever() 
llm = AzureChatOpenAI(deployment_name="gpt-4", temperature=0)
query = RetrievalQA.from_chain_type(llm = llm, chain_type = "stuff", retriever = retriever, \
                                return_source_documents=True)

# Pass the query and the retrieved/relevant data to the LLM
q1_name = query(f"What is the age and gender of the patient {pat_num}?\
                List any medications taken.\
                List any current symptions mentioned by this patient.\
                List any family medical history hx.\
                Provide your answers as a markdown")
print(q1_name["result"])
q1_name["source_documents"]

- Age and gender of patient 81630: 67-year-old female
- Medications taken: HCTZ, Lisinopril
- Current symptoms mentioned by the patient: Insomnia for 3 weeks, difficulty falling asleep, tossing and turning, awakening early, sadness, decreased interest in activities, low energy
- Family medical history: Mother with a history of depression, currently in her 90s; no history of thyroid issues

Please note that this information is provided as a markdown.


[Document(page_content="pn_num: 81630\ncase_num: 8\npn_history: 67 yo female with hx of breast CA in remission and HTN presents to clinic with c/o 3 weeks of insomnia. Pt states her son died suddenly in late July in an MVA. Since that time, she has found it difficult to fall asleep and also feels as though she tosses and turns and awakens early. She tried a friend's Ambien for the past 5 nights without relief. Of note, pt has felt sad and notes decreased interest in activities as well as low energy. However, she denies SI, HI, depression/hopelessness, guilt, concentration defitict, or psychomotor agitation or depression. Denies palpitations, chest pain, cough, SOB, HA, numbness, tingling or weakness. \r\nROS: as above\r\nPMH: HTN, breast cancer in remission x10 years\r\nFam: Mother hx of depression, in 90s now. No hx of thyroid \r\nSurg: Lumpectomy, laparotomy for appendix rupture in 20s\r\nSoc: Never smoker. 2 glasses of wine 2-3x per week, none in past 3 weeks. No illicit drugs Meds:

In [271]:
pat_num = '45368'

# Retrieve rows for this patient only, from the vector store
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import RetrievalQA
retriever = full_index.as_retriever(search_type="mmr", search_kwargs={'filter': {'source': pat_num}}) 
llm = AzureChatOpenAI(deployment_name="gpt-4", temperature=0)
query = RetrievalQA.from_chain_type(llm = llm, chain_type = "stuff", retriever = retriever, \
                                return_source_documents=True)

# Pass the query and the retrieved/relevant data to the LLM
q1_name = query(f"What is the age and gender of the patient {pat_num}?\
                List any medications taken.\
                List any current symptions mentioned by this patient.\
                List any family medical history hx.\
                Provide your answers as a markdown")
print(q1_name["result"])
q1_name["source_documents"]

- Age and Gender: 45-year-old female (45 yo F)
- Medications: Tylenol
- Current Symptoms: Nervousness, anxiety (especially on Sunday nights and Monday mornings), difficulty falling asleep, decreased appetite
- Family Medical History: Patient's dad died of a heart attack at 65. Mother is alive.


[Document(page_content='pn_num: 45368\ncase_num: 4\npn_history: 45 yo F presents with chief complaint of "nervousness." Patient states that she has been "anxious all the time" since she switched to a lecturing position at her teaching job a few weeks ago. Anxiety is worst on Sunday nights and Monday mornings. Patient endorses associated difficulty falling asleep without difficulty maintaining sleep, as well as decreased appetite. Patient states that she drinks 5-6 cups of coffee per day, but states that she has done this for years. Denies depressed mood, feelings of guilt/worthlessness, loss of energy, difficulty concentrating, suicidal ideation, visual and auditory hallucinations\r\nPMH: Denies\r\nPSH: Denies\r\nFH: Patient\'s dad died of a heart attack at 65. Mother alive \r\nSH: Patient drinks alcohol very occasionally, denies smoking, illicit drug use. Sexually active with husband, does not use contraception. Drinks 5-6 cups of coffee per day\r\nMeds: Tylenol\r\nAllergies: NKA', me

In [272]:
pat_num = '22006'

# Retrieve rows for this patient only, from the vector store
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import RetrievalQA
retriever = full_index.as_retriever(search_type="mmr", search_kwargs={'filter': {'source': pat_num}}) 
#retriever = full_index.as_retriever() 
llm = AzureChatOpenAI(deployment_name="gpt-4", temperature=0)
query = RetrievalQA.from_chain_type(llm = llm, chain_type = "stuff", retriever = retriever, \
                                return_source_documents=True)

# Pass the query and the retrieved/relevant data to the LLM
q1_name = query(f"What is the age and gender of the patient {pat_num}?\
                List any medications taken.\
                List any current symptions mentioned by this patient.\
                List any family medical history hx.\
                Provide your answers as a markdown")
print(q1_name["result"])
q1_name["source_documents"]

- Age and gender of patient 22006: 44 years old female
- Medications taken: HCTZ
- Current symptoms mentioned by the patient: irregular periods, heavy menstrual bleeding (using 2-3 pads and at times 6 pads), sweating, and hot flashes
- Family medical history: mother with osteoarthritis (OA)


[Document(page_content='pn_num: 22006\ncase_num: 2\npn_history: 44 yof presents with irregular periods. Patient states it started 3 yrs ago now her last menses was 2 months ago using 2-3 pads and at times 6 pads when heavy. she denies any discharge, odor, bleeding between menses, change in weight, feeling cold or hot, diarrehea or constipation, abdominal pain change in urine or bowel but complains of sweating and hot flashes recently.\r\nROS: negative except as above\r\nPMHX:HTN x 6 yrs, FMHX: mother with OA, meds: HCTZ, NKDA, PSH: none, SH: denies smoke, drugs, social drinker\r\nOBGYN: G2P2, sexually active with husband, denies any STD. Last pap smear was 1 year ago and normal.', metadata={'source': '22006', 'row': 100})]

In [273]:
pat_num = '36193'

# Retrieve rows for this patient only, from the vector store
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import RetrievalQA
retriever = full_index.as_retriever(search_type="mmr", search_kwargs={'filter': {'source': pat_num}}) 
#retriever = full_index.as_retriever() 
llm = AzureChatOpenAI(deployment_name="gpt-4", temperature=0)
query = RetrievalQA.from_chain_type(llm = llm, chain_type = "stuff", retriever = retriever, \
                                return_source_documents=True)

# Pass the query and the retrieved/relevant data to the LLM
q1_name = query(f"What is the age and gender of the patient {pat_num}?\
                List any medications taken.\
                List any current symptions mentioned by this patient.\
                List any family medical history hx.\
                Provide your answers as a markdown")
print(q1_name["result"])
q1_name["source_documents"]

- Age and Gender: 35 yo Male (35 yo M)
- Medications: Motrin, Antacid
- Current Symptoms: Epigastric pain (burning, gnawing), intermittent, not associated with food intake, worsening frequency (previously 1x week, now 2/3x day), fatigue, decreased appetite, bloating
- Family Medical History: Bleeding ulcer (father)


[Document(page_content="pn_num: 36193\ncase_num: 3\npn_history: Px is 35 yo M presenting with epigastric pain 2 mo's duration.\r\n-Burning, gnawing\r\n-Intermittent, no associated with food intake\r\n-Previous 1x week, now 2/3x day, worsening\r\n-Antacids previously alleviated pain\r\n+fatigue, decreased appetite, bloating\r\n\r\nROS: neg except as above\r\nPMH: back pain\r\nPSH: none\r\nAllergies: NKDA\r\nMeds: motrin, antacid\r\nSH: 1 PPD since 15 yo, 3-4x beers /week, fast food diet\r\nFH: bleeding ulcer - father", metadata={'source': '36193', 'row': 2})]

In [241]:
data[99]

Document(page_content='pn_num: 55890\ncase_num: 5\npn_history: Ms Whelan is a 26 yo F with no significant PMHx, coming today in today for follow for ED visit for palpitations. about 5 years prior Ms Whelan had her first episode of heart palpitations however they were infrequent until 3 weeks ago when they started occuring 1-2x/day, and cooccured with finger numbness. She then visited the ED where EKG, CMP, cardiac enzymes, and EKg were normal. since then she has had palpitations about 1x/day, but no finger numbness. During these episodes she gets SOB and her throat "closes up", she also feels hot and clammy, and nauseous. Of note she did recently bought a condo 3months ago and then got laid off. She exercises regularly.\r\nPMHx: followed by and OBGYN and has no current problems. regular menstural cycle. no past surgeries or allergies\r\nFMHx: none\r\nSocHx: used to work in sales. no tobacco or etoh or drug use. No caffiene use. sexually active with 1 male, vaginal sex, uses condoms', m

In [340]:
pat_num = '55890'

from langchain.chat_models import AzureChatOpenAI
from langchain.chains import RetrievalQA
#retriever = full_index.as_retriever(search_kwargs={'filter': {'source': pat_num}}) 
retriever = full_index.as_retriever(search_type="mmr", search_kwargs={'filter': {'source': pat_num}}) 
llm = AzureChatOpenAI(deployment_name="gpt-4", temperature=0)
query = RetrievalQA.from_chain_type(llm = llm, chain_type = "stuff", retriever = retriever, \
                                return_source_documents=True)

# Pass the query and the retrieved/relevant data to the LLM
#q1_name = query(f"What is the age and gender of this patient {pat_num}?")
q1_name = query(f"What is the age and gender of this patient {pat_num}?\
                List any medications taken.\
                List any current symptions mentioned by this patient.\
                List any family medical history hx.\
                Provide your answers as a markdown")
print(q1_name["result"])
q1_name["source_documents"]

I'm sorry, but I don't have any information about patient 55890. I cannot provide any details about their age, gender, medications, symptoms, or family medical history.


[]

In [339]:
pat_num = '1649'

from langchain.chat_models import AzureChatOpenAI
from langchain.chains import RetrievalQA
retriever = full_index.as_retriever(search_type="mmr", search_kwargs={'filter': {'source': pat_num}}) 
#retriever = full_index.as_retriever() 
llm = AzureChatOpenAI(deployment_name="gpt-4", temperature=0)
query = RetrievalQA.from_chain_type(llm = llm, chain_type = "stuff", retriever = retriever, \
                                return_source_documents=True)

# Pass the query and the retrieved/relevant data to the LLM
q1_name = query(f"What is the age and gender of this patient {pat_num}?\
                List any medications taken.\
                List any current symptions mentioned by this patient.\
                List any family medical history hx.\
                Provide your answers as a markdown")
print(q1_name["result"])
q1_name["source_documents"]

- Age and Gender: 17 y.o. Male
- Medications: Takes Adderall few days a week
- Current Symptoms: Intermittent episodes of 'heart pounding' for the past 6 months, worsened over time, chest pressure without pain, no radiation, no headaches, shortness of breath and near syncope during a recent soccer game
- Family Medical History: Mother with thyroid condition, Father had a heart attack 1 year ago (age 52)


[Document(page_content="pn_num: 1649\ncase_num: 0\npn_history: Mr. Cleveland is a 17 y.o. M presenting w/ intermittent episodes of 'heart pounding' for the past 6 months. This worsened over time and is without a trigger. Episodes come and go awa on their own. chest pressure without pain, no radiation. Denies headaches. Most recent episode was 2 days ago during soccer game when he had shortness of breath and near syncope. He is worried he will not play in tomorrow's game. \r\nPMH- None\r\nPSH- non\r\nMeds- Takes adderall few days a week\r\nAllergies- none\r\nMeds0- mother w/ thyroid condition. Father w/ heart attack 1 year ago (age 52)\r\nSocial- Lives in dorm. Freshman in colleg. Drinks weekends 3-4 beers. CAGE - Tried marijuana once. Sexually active w/ girlfriend. Uses condoms.", metadata={'source': '1649', 'row': 98})]